In [ ]:
# setup paths
import sys
basedir = '/Users/nickm/thesis/icml2023paper/1d_euler'
readwritedir = '/Users/nickm/thesis/icml2023paper/1d_euler'

sys.path.append('{}/core'.format(basedir))
sys.path.append('{}/simulate'.format(basedir))
sys.path.append('{}/ml'.format(basedir))

In [ ]:
# import external packages
import jax
import jax.numpy as jnp
import numpy as onp
from jax import config, vmap
config.update("jax_enable_x64", True)
import xarray
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
from initialconditions import get_a0, f_init_lax_shock_tube
from simparams import CoreParams
from helper import get_rho, get_u, get_p

In [ ]:
def get_core_params(Lx = 1.0, gamma = 5/3, bc = 'periodic', fluxstr = 'laxfriedrichs'):
    return CoreParams(Lx, gamma, bc, fluxstr)



def plot_a(a, core_params):
    fig, axs = plt.subplots(1, 3, figsize=(11, 3))
    axs[0].plot(get_rho(a, core_params))
    axs[0].set_title(r'$\rho$')
    
    axs[1].plot(get_u(a, core_params))
    axs[1].set_title(r'$u$')
    
    axs[2].plot(get_p(a, core_params))
    axs[2].set_title(r'$p$')

In [ ]:
kwargs_core_params = {'Lx': 10.0, 'gamma': 1.4, 'bc': 'ghost', 'fluxstr': 'laxfriedrichs'}

core_params = get_core_params(**kwargs_core_params)
f_init = f_init_lax_shock_tube(core_params)

In [ ]:
nx = 100
a0 = get_a0(f_init, core_params, nx)

In [ ]:
plot_a(a0, core_params)